<a href="https://colab.research.google.com/github/JonathanFly/stylegan-art/blob/master/Render_Videos_of_All_StyleGAN_Models_in_GDrive_Folder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**What is This**

Do you have StyleGAN models scattered all over your personal computer or Google Drive?  Do you wonder if you might have a cool checkpoint or just what the heck these models even are?  

This is a stripped down version of [@roadrunning01](https://github.com/ak9250/stylegan-art)'s StyleGAN Colab notebook that does one thing: it takes every StyleGAN model in Google Drive folder and renders out a video interpolation of each, then puts these .mp4 files in that sub folder of that Google Drive folder.

**Setup**

1) Make sure GPU is enabled, go to edit->notebook settings->Hardware Accelerator GPU

2) Make a copy to your google drive, click on copy to drive in panel.

3) Put al the StyleGAN .pkl models you want to render in a single directory on your Google Drive.

3) Edit the cell with **gdrive_content_path** with the path to your StyleGAN models on your google drive.

4) Run the notebook, and be sure to wait until you get the prompt to mount your Google Drive and copy your key into the field that pops up.

**What happens when Google inevitably and unpredictably yanks the plug in the middle of this notebook running?**

Just run the notebook again. Each video is copied to your Google drive immediately after rendering, and during the render step it will skip that model if it already has an .mp4 of the same name in the output folder. So just keep running the notebook until it finishes the directory.




In [0]:
gdrive_content_path = '/content/gdrive/My Drive/StyleGAN_Models/'

In [0]:
gdrive_content_filter = '*.pkl'

In [0]:
gdrive_output_path = gdrive_content_path + '/video_interpolations/'

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!mkdir /content/StyleGAN_local_models

In [0]:
!mkdir "{gdrive_output_path}"

In [0]:
#This could check if the .mp4 files already exist in the output path before copying to save time when resuming...
%cp -r "{gdrive_content_path}"{gdrive_content_filter} /content/StyleGAN_local_models/

In [0]:
!git clone https://github.com/NVlabs/stylegan.git

In [0]:
cd stylegan/

In [0]:
import os
import pickle
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config
import moviepy.editor as mpy
from tqdm import tqdm_notebook as tqdm
tflib.init_tf()

In [0]:
truncation = 0.7


def bookmark(latents, new_faves):
    for f in new_faves:
        faves.append(latents[f])

def show_faves(faves):
    latents = np.array(faves)
    labels = np.zeros([latents.shape[0]] + Gs.input_shapes[1][1:])
    n = len(faves)
    nr, nc = math.ceil(n / 6), 6
    for r in range(nr):
        images = Gs.run(latents[6*r:min(n-1, 6*(r+1))], None, truncation_psi=truncation, randomize_noise=False, output_transform=fmt)
        img1 = np.concatenate([img for img in images], axis=1)
        plt.figure(figsize=(24,4))
        plt.imshow(img1)
        
def random_sample(num_images, scale):
    latents = np.random.RandomState(int(1000*random.random())).randn(num_images, *Gs.input_shapes[0][1:])
    labels = np.zeros([latents.shape[0]] + Gs.input_shapes[1][1:])
    images = Gs.run(latents, None, truncation_psi=truncation, randomize_noise=False, output_transform=fmt)
    images_ct = np.concatenate([img for img in images], axis=1)
    plt.figure(figsize=(scale*num_images, scale))
    plt.imshow(images_ct)
    return images, latents

def get_latent_interpolation(endpoints, num_frames_per, mode, shuffle):
    if shuffle:
        random.shuffle(endpoints)
    num_endpoints, dim = len(endpoints), len(endpoints[0])
    num_frames = num_frames_per * num_endpoints
    endpoints = np.array(endpoints)
    latents = np.zeros((num_frames, dim))
    for e in range(num_endpoints):
        e1, e2 = e, (e+1)%num_endpoints
        for t in range(num_frames_per):
            frame = e * num_frames_per + t
            r = 0.5 - 0.5 * np.cos(np.pi*t/(num_frames_per-1)) if mode == 'ease' else float(t) / num_frames_per
            latents[frame, :] = (1.0-r) * endpoints[e1,:] + r * endpoints[e2,:]
    return latents

def get_latent_interpolation_bspline(endpoints, nf, k, s, shuffle):
    if shuffle:
        random.shuffle(endpoints)
    x = np.array(endpoints)
    x = np.append(x, x[0,:].reshape(1, x.shape[1]), axis=0)
    nd = x.shape[1]
    latents = np.zeros((nd, nf))
    nss = list(range(1, 10)) + [10]*(nd-19) + list(range(10,0,-1))
    for i in tqdm(range(nd-9)):
        idx = list(range(i,i+10))
        tck, u = interpolate.splprep([x[:,j] for j in range(i,i+10)], k=k, s=s)
        out = interpolate.splev(np.linspace(0, 1, num=nf, endpoint=True), tck)
        latents[i:i+10,:] += np.array(out)
    latents = latents / np.array(nss).reshape((512,1))
    return latents.T


def generate_images(latents, labels):
    batch_size = 8
    num_frames = latents.shape[0]
    num_batches = int(np.ceil(num_frames/batch_size))
    images = []
    for b in tqdm(range(num_batches)):
        new_images = Gs.run(latents[b*batch_size:min((b+1)*batch_size, num_frames-1), :], None, truncation_psi=truncation, randomize_noise=False, output_transform=fmt)
        for img in new_images:
            images.append(img)
    return images

def make_movie(images, out_dir, out_name):
    temp_dir = 'frames%06d'%int(1000000*random.random())
    os.system('mkdir %s'%temp_dir)
    for idx in tqdm(range(len(images))):
        PIL.Image.fromarray(images[idx], 'RGB').save('%s/frame%05d.png' % (temp_dir, idx))
    cmd = 'ffmpeg -loglevel error -i %s/frame%%05d.png -c:v libx264 -pix_fmt yuv420p "%s/%s.mp4"' % (temp_dir, out_dir, out_name)
    print(cmd)
    os.system(cmd)
    os.system('rm -rf %s'%temp_dir)
    
def random_sample(num_images, scale):
    latents = np.random.RandomState(int(1000*random.random())).randn(num_images, *Gs.input_shapes[0][1:])
    labels = np.zeros([latents.shape[0]] + Gs.input_shapes[1][1:])
    images = Gs.run(latents, None, truncation_psi=truncation, randomize_noise=False, output_transform=fmt)
    images_ct = np.concatenate([img for img in images], axis=1)
    plt.figure(figsize=(scale*num_images, scale))
    #plt.imshow(images_ct)
    plt.axis('off')
    #plt.savefig('download.png')
    return images, latents

In [0]:
import shutil
import os.path

path, dirs, files = next(os.walk('/content/StyleGAN_local_models/'))
total_files = len(files)

print("Files In Directory: " + str(total_files))

file_count = 0

for filename in os.listdir('/content/StyleGAN_local_models/'):
    file_count = file_count + 1
    print("File: " + str(file_count) + " of " + str(total_files) + " (" + filename + ")")
    if filename.endswith("pkl"):
      
      output_movie_filename = filename.replace(' ','_'); 
      
      if os.path.exists(gdrive_output_path + output_movie_filename + 'easemovie' + '.mp4') == True: 
        print("File: " + output_movie_filename + 'easemovie' + '.mp4' + " already exists. Skipping.")
        continue
        
      model = '/content/StyleGAN_local_models/' + filename
      with open(model, 'rb') as f:
        _G, _D, Gs = pickle.load(f)
        
      fmt = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
      synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=8)
         
      #images, latents = random_sample(40, scale=8)
      #latents = get_latent_interpolation(latents, 45, 'linear', False)
      #labels = np.zeros([latents.shape[0]] + Gs.input_shapes[1][1:])
      #images = generate_images(latents, labels)    
      #make_movie(images, '/content/stylegan',  output_movie_filename + 'linearmovie')
      #shutil.copyfile(output_movie_filename + 'linearmovie' + '.mp4', gdrive_output_path + output_movie_filename + 'linearmovie' + '.mp4') 
 
      images, latents = random_sample(40, scale=8)
      latents = get_latent_interpolation(latents, 45, 'ease', False)
      labels = np.zeros([latents.shape[0]] + Gs.input_shapes[1][1:])
      images = generate_images(latents, labels)    
      make_movie(images, '/content/stylegan', output_movie_filename + 'easemovie')
      shutil.copyfile(output_movie_filename + 'easemovie' + '.mp4', gdrive_output_path + output_movie_filename + 'easemovie' + '.mp4') 
  

In [0]:
print("Finished: " + str(total_files) + " files processed")